In [ ]:
import os
import subprocess

# Set file paths and directories
input_dir = "/Users/huyenphung/Desktop/backend/video/"
output_base_dir = "/Users/huyenphung/Desktop/backend/keyframe_infomaton/keyframe/"

# Loop through each video in the input directory
for input_video in os.listdir(input_dir):
    if input_video == ".DS_Store":
        continue
    input_video_path = os.path.join(input_dir, input_video)
    video_name = os.path.splitext(input_video)[0]
    output_dir = os.path.join(output_base_dir, video_name)
    
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Get the keyframe timestamps using ffprobe
    ffprobe_command = [
        "/Users/huyenphung/Downloads/ffprobe", "-loglevel", "error", 
        "-select_streams", "v:0", "-show_entries", "packet=pts_time,flags", 
        "-of", "csv=print_section=0", input_video_path
    ]

    # Run the ffprobe command and capture the output
    ffprobe_result = subprocess.run(ffprobe_command, stdout=subprocess.PIPE, text=True)

    # Filter the keyframes (lines containing ",K")
    keyframe_lines = [line.split(",")[0] for line in ffprobe_result.stdout.splitlines() if ",K" in line]

    # Extract keyframes using ffmpeg
    for index, timestamp in enumerate(keyframe_lines, start=1):
        output_image = os.path.join(output_dir, f"keyframe_{index}_{timestamp}.jpg")
        
        ffmpeg_command = [
            "ffmpeg", "-loglevel", "error", 
            "-ss", timestamp, "-i", input_video_path, 
            "-frames:v", "1", output_image
        ]
        
        subprocess.run(ffmpeg_command)

    print(f"Keyframes extracted and saved to {output_dir} for {input_video}")


In [3]:
import os
import math
from PIL import Image

def reduce_frame(root, new_root, reduce_ratio=0.5):
    for folder in os.listdir(root):
        folder_path = os.path.join(root, folder)
        new_dir = os.path.join(new_root, f"{folder}_reduced")
        
        # Ensure we're only processing directories
        if not os.path.isdir(folder_path):
            continue

        if not os.path.exists(new_dir):
            os.makedirs(new_dir)

        for frame_path in os.listdir(folder_path):
            full_frame_path = os.path.join(folder_path, frame_path)
            
            # Ensure we're only processing files and .jpg images
            if os.path.isfile(full_frame_path) and frame_path.endswith('.jpg'):
                frame_name = frame_path.replace('.jpg', '.webp')
                image = Image.open(full_frame_path)
                
                # Get the original size and calculate the new size
                x, y = image.size
                x_new, y_new = math.floor(x * reduce_ratio), math.floor(y * reduce_ratio)
                
                # Resize and save the image in .webp format
                image = image.resize((x_new, y_new))
                image.save(f'{new_dir}/{frame_name}', optimize=True, quality=30, format="webp")
                print(f"Saved: {new_dir}/{frame_name}")

In [ ]:
# Define root paths
root = "/Users/huyenphung/Desktop/backend/keyframe_infomaton/keyframe"
new_root = "/Users/huyenphung/Desktop/backend/keyframe_infomaton/keyframe"

# Call the function
reduce_frame(root, new_root)